Tento notebook slouží k **ověření, že máte nainstalované potřebné knihovny** a že vám funguje připojení k webovým stránkám.

**Prosím spusťte všechny buňky postupně a ověřte, že vám vše funguje.**

# Session 4 - Uvod do Web Scrapingu

V predchozich lekcich jsme se naucili ziskavat data z webovych stranek pomoci jejich API. Ne kazda stranka ma API - mnohe z nich byly vytvoreny s predpokladem, ze jejich obsah budou cist pouze lide. To ale neznamena, ze nelze tento obsah ziskat pomoci Pythonu - jen je potreba pouzit jine nastroje.

**Web scraping** je technika pro stahovani a extrahovani informaci z webovych stranek. Sklada se z:
1. Stazeni obsahu webove stranky (napr. pomoci knihovny `requests`)
2. Transformace obsahu (stazeny obsah je jeden dlouhy retezec) do struktury, ve ktere se da lepe vyhledavat
3. Extrakce potrebnych informaci

V teto lekci se naucime:
- Zaklady knihovny `requests` pro stahovani webovych stranek
- Parsovani HTML pomoci `BeautifulSoup`
- CSS selektory pro vyhledavani elementu

## Proc je web scraping obtizny?

Ve srovnani se ziskavanim dat pres API ma web scraping vyznamne nevyhody:

**1. Musite dobre znat strukturu stranky** - Neexistuje zadna dokumentace. Sami si musite zjistit, jak vypadaji odkazy a jak je obsah stranky rozlozen.

**2. Data nejsou tak strukturovana jako v API** - Obsah stranky v HTML je podrizen tomu, jak ma stranka vypadat, ne tomu, jak se data snadno zpracuji.

**3. Stranka se muze zmenit bez varovani** - Autor stranky nezarucuje, ze stranka bude vzdy vypadat stejne. Skript muze prestat fungovat a bude potrebovat opravy.

**4. Stranka muze byt dynamicka** - JavaScript muze nacitat obsah az po nacteni stranky. V takovem pripade je potreba pouzit nastroje jako Selenium.

## Struktura URL adresy

Kazda webova stranka se nachazi na nejake adrese. Priklad:

`http://host.name.eu:80/directory/index.html?param1=value1&param2=value2`

Casti URL adresy:
1. `http://` nebo `https://` - protokol (`https` je sifrovana verze `http`)
2. `host.name.eu` - domenove jmeno serveru
3. `:80` - cislo portu (vychozi: 80 pro http, 443 pro https)
4. `/directory/index.html` - cesta k souboru na serveru
5. `?param1=value1&param2=value2` - querystring: dodatecne parametry (zacinaji `?`, oddelene `&`)

Priklad moderni URL: `https://example.com/article/2024/05/my-article?ref=newsletter`

---

## 1. Instalace knihovny
Nejprve musime nainstalovat potrebne knihovny. Pouzijeme `pip` primo v notebooku.

### Proc pouzivame vykricnik `!` ?

V Jupyter notebooku se kod interpretuje jako Python. Pro prikazy prikazoveho radku (terminalu) pouzivame `!` na zacatku.

```python
print('Ahoj')       # Python kod - funguje primo
!pip install requests  # Prikaz pro terminal - potrebuje !
```

### `!pip` vs `%pip`

| Prikaz | Kam instaluje |
|--------|---------------|
| `!pip install` | Podle systemoveho PATH (muze byt jine prostredi) |
| `%pip install` | Vzdy do prostredi aktualniho kernelu |

Doporuceni: V Anaconde je bezpecnejsi pouzivat `%pip`, aby se balicek urcite nainstaloval do spravneho prostredi.

---

## 2. Test importu knihoven

Pokud import proběhne bez chyby, knihovny jsou správně nainstalovány.

In [ ]:
import requests
from bs4 import BeautifulSoup

print("Import knihoven proběhl úspěšně!")

---

## 3. Test připojení - Bankier.pl

Tento příklad ukazuje stažení stránky polské banky. Stránka byla **redesignována**, takže se na ni dostaneme (stavový kód 200), ale původní element `boxKursyNbpTab1` tam již není.

**Očekávaný výsledek:** Stavový kód 200, ale 0 nalezených kurzů.

In [ ]:
# Test připojení k Bankier.pl
website = requests.get('https://www.bankier.pl/waluty')

print(f"Stavový kód: {website.status_code}")
print(f"Připojení úspěšné: {website.ok}")

In [ ]:
# Hledáme element, který tam dříve byl (po redesignu už neexistuje)
content = BeautifulSoup(website.text, 'html.parser')
div_rates = content.find('div', id='boxKursyNbpTab1')

if div_rates:
    print("Element nalezen - stránka nebyla změněna.")
else:
    print("Element NENALEZEN - stránka byla redesignována.")
    print("Toto je očekávané chování. Na lekci si ukážeme, proč k tomu dochází.")

---

## 4. Test funkčního příkladu - Quotes to Scrape

Tento příklad by měl fungovat správně. Stránka `quotes.toscrape.com` je speciálně vytvořená pro výuku webscrapingu.

**Očekávaný výsledek:** Stavový kód 200, nalezeno 10 citátů.

In [ ]:
# Stažení stránky
url = "https://quotes.toscrape.com"
response = requests.get(url)

print(f"Stavový kód: {response.status_code}")
print(f"Připojení úspěšné: {response.ok}")

In [ ]:
# Parsování a nalezení citátů
soup = BeautifulSoup(response.text, 'html.parser')
quotes = soup.find_all('div', class_='quote')

print(f"Nalezeno citátů: {len(quotes)}")

In [ ]:
# Ukázka - zobrazení prvních 3 citátů
for i, quote in enumerate(quotes[:3], 1):
    text = quote.find('span', class_='text').text
    author = quote.find('small', class_='author').text
    
    print(f"{i}. {author}:")
    print(f"   {text}")
    print()

---

## Shrnutí

Pokud jste viděli:

- **Import proběhl úspěšně** ✓
- **Bankier.pl: Stavový kód 200, element NENALEZEN** ✓
- **Quotes: Stavový kód 200, nalezeno 10 citátů** ✓

...pak je vše v pořádku a jste připraveni na lekci!

Na lekci si vše projdeme detailně.